
<a href="https://colab.research.google.com/github/NeuroDesk/example-notebooks/blob/main/bids_conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Google Colab"/>   </a>

# BIDS

Author: Thuy Dao

Citation:
- dcm2niix: https://github.com/rordenlab/dcm2niix/
- bidscoin: https://doi.org/10.3389/fninf.2021.770608
- heudiconv: https://doi.org/10.5281/zenodo.1012598

## Setup Neurodesk

In [1]:
%%capture
import os
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
  os.environ["LD_PRELOAD"] = "";
  os.environ["APPTAINER_BINDPATH"] = "/content,/tmp,/cvmfs"
  os.environ["MPLCONFIGDIR"] = "/content/matplotlib-mpldir"
  os.environ["LMOD_CMD"] = "/usr/share/lmod/lmod/libexec/lmod"

  !curl -J -O https://raw.githubusercontent.com/NeuroDesk/neurocommand/main/googlecolab_setup.sh
  !chmod +x googlecolab_setup.sh
  !./googlecolab_setup.sh

  os.environ["MODULEPATH"] = ':'.join(map(str, list(map(lambda x: os.path.join(os.path.abspath('/cvmfs/neurodesk.ardc.edu.au/neurodesk-modules/'), x),os.listdir('/cvmfs/neurodesk.ardc.edu.au/neurodesk-modules/')))))

In [2]:
# Output CPU information:
!cat /proc/cpuinfo | grep 'vendor' | uniq
!cat /proc/cpuinfo | grep 'model name' | uniq

vendor_id	: AuthenticAMD
model name	: AMD EPYC 7742 64-Core Processor


## Data preparation

Download DICOM data from OSF

In [17]:
!pip install osfclient
!osf -p ru43c clone . > /dev/null 2>&1
!tar xf osfstorage/dicoms-unsorted.tar
!rm -rf osfstorage/
!tree dicoms-unsorted | head
!echo -e "...\nThere are `ls dicoms-unsorted | wc -l` unsorted DICOMs in ./dicoms-unsorted/"

dicoms-unsorted
├── MR.1.1.dcm
├── MR.1.10.dcm
├── MR.1.100.dcm
├── MR.1.101.dcm
├── MR.1.102.dcm
├── MR.1.103.dcm
├── MR.1.104.dcm
├── MR.1.105.dcm
├── MR.1.106.dcm
...
There are 1216 unsorted DICOMs in ./dicoms-unsorted/


In [13]:
!datalad install https://datasets.datalad.org/repronim/heudiconv-tutorial-example
!cd heudiconv-tutorial-example/MRIS/dicom/219/itbs && datalad get Bzero_verify_PA_17

Cloning: 100%|██████████████████████| 2.00/2.00 [00:01<00:00, 1.46 candidates/s]
Enumerating: 0.00 Objects [00:00, ? Objects/s]
                                              
Counting:   0%|                              | 0.00/25.9k [00:00<?, ? Objects/s]
Counting:  39%|████████▌             | 10.1k/25.9k [00:00<00:00, 101k Objects/s]
                                                                                
Compressing:   0%|                           | 0.00/13.6k [00:00<?, ? Objects/s]
Compressing:  61%|██████████▉       | 8.29k/13.6k [00:00<00:00, 75.9k Objects/s]
                                                                                
Receiving:   0%|                             | 0.00/25.9k [00:00<?, ? Objects/s]
Receiving:   1%|▏                     | 259/25.9k [00:00<00:23, 1.10k Objects/s]
Receiving:   3%|▋                     | 776/25.9k [00:00<00:09, 2.60k Objects/s]
Receiving:   5%|█                   | 1.29k/25.9k [00:00<00:07, 3.36k Objects/s]
Receiving:   8%

In [ ]:
from ipyniivue import AnyNiivue
nv = AnyNiivue()
nv.load_volumes([{"path": "./ds000102/sub-08/anat/sub-08_T1w.nii.gz"}])
nv

## Convert DICOMS to BIDS-formatted NIFTI

### 1.1. Using dcm2niix (manual workflow)

In [10]:
import lmod
await lmod.load('dcm2niix/v1.0.20240202')

In [15]:
!mkdir dcm2niix_bids
!dcm2niix n -o ./dcm2niix_bids ./heudiconv-tutorial-example/MRIS/dicom/219/itbs/Bzero_verify_PA_17

mkdir: cannot create directory ‘dcm2niix_bids’: File exists
Chris Rorden's dcm2niiX version v1.0.20240202  (JP2:OpenJPEG) (JP-LS:CharLS) GCC8.4.0 x86-64 (64-bit Linux)
Found 148 DICOM file(s)
Convert 148 DICOM as ./dcm2niix_bids/Bzero_verify_PA_17_Bzero_verify_P-A_20180706110327_17 (128x128x74x2)
Conversion required 5.090344 seconds (0.370856 for core code).


### 1.2. Using bidscoin (automated workflow)

In [5]:
await lmod.load('bidscoin/4.3.3')

In [7]:
!dicomsort ./dicoms-unsorted
!bidsmapper ./dicoms-unsorted ./bidscoin_bidsmap    # Scans your data and creates a study bidsmap
!bidscoiner ./bidscoin_bidsmap ./bidscoin_bids    # Converts your data to BIDS using the study bidsmap

INFO | 
INFO | -------------- START BIDSmapper ------------
INFO | >>> bidsmapper sourcefolder=/data/books/workflows/dicoms-unsorted bidsfolder=/data/books/workflows/bidscoin_bidsmap bidsmap=bidsmap.yaml template=/home/jovyan/.bidscoin/4.3.3+qt5/templates/bidsmap_dccn.yaml plugins=[] subprefix=None sesprefix=None store=False force=False
INFO | No existing bidsmap file found: /data/books/workflows/bidscoin_bidsmap/code/bidscoin/bidsmap.yaml
INFO | Reading: /home/jovyan/.bidscoin/4.3.3+qt5/templates/bidsmap_dccn.yaml
INFO | Checking the bidsmap run-items:
SUCCESS | All datatypes and options in the template bidsmap are valid
WARNING | No subjects found in: /data/books/workflows/dicoms-unsorted/sub-*
INFO | Opening the bidseditor
qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this prob

### 1.3. Using HeuDiConv (automated workflow)

heudiconv provides over 10 pre-created heuristics, which can be seen here .

These heuristic files are documented in their code comments. Some of them, like convertall or ReproIn could be immediately reused and represent two ends of the spectrum in heuristics:- `

conver`tall is very simple and does not automate anything – it is for a user to modify filenames in the prepared conversion table, and then rerun with -c dcm2nii- `x.

r`eproin can be used fully automated, if original sequences were named according to its ReproIn convention.

In [2]:
await lmod.load('heudiconv/1.3.1')

In [16]:
!heudiconv -f convertall -s 219 -c dcm2niix -b -o ./heudiconv_bids --files ./heudiconv-tutorial-example/MRIS/dicom/

INFO: Running heudiconv version 1.3.1 latest 1.3.2
INFO: Analyzing 1716 dicoms
Traceback (most recent call last):
  File "/opt/miniconda-py39_4.12.0/bin/heudiconv", line 8, in <module>
    sys.exit(main())
  File "/src/heudiconv/heudiconv/cli/run.py", line 30, in main
    workflow(**kwargs)
  File "/src/heudiconv/heudiconv/main.py", line 410, in workflow
    study_sessions = get_study_sessions(
  File "/src/heudiconv/heudiconv/parser.py", line 221, in get_study_sessions
    seqinfo_dict = group_dicoms_into_seqinfos(
  File "/src/heudiconv/heudiconv/dicoms.py", line 338, in group_dicoms_into_seqinfos
    mwinfo = validate_dicom(filename, dcmfilter)
  File "/src/heudiconv/heudiconv/dicoms.py", line 171, in validate_dicom
    mw = dw.wrapper_from_file(fl, force=True, stop_before_pixels=True)
  File "/opt/miniconda-py39_4.12.0/lib/python3.9/site-packages/nibabel/nicom/dicomwrappers.py", line 59, in wrapper_from_file
    with ImageOpener(file_like) as fobj:
  File "/opt/miniconda-py39_4.12.

## Process data

### Brain extraction using FSL BET

In [ ]:
await lmod.load('fsl/6.0.7.4')
await lmod.list()

In [ ]:
%% bash

declare -a niifile=($(find "$PWD" -type f -iname "*.nii"));
for filename in "${niifile[@]}";
    do 
        echo ${filename}
        echo "running BET"
        bet ${filename}.nii ${filename%.*}_brain
    done

### Deface

## Share your data to OpenNeuro
